# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902056


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:06<01:34,  3.38s/it]

Rendering models:  10%|█         | 3/30 [00:06<01:04,  2.41s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:44,  1.73s/it]

Rendering models:  20%|██        | 6/30 [00:07<00:29,  1.23s/it]

...

Rendering models:  80%|████████  | 24/30 [00:08<00:00,  7.57it/s]

Rendering models:  87%|████████▋ | 26/30 [00:09<00:00,  7.97it/s]

Rendering models:  93%|█████████▎| 28/30 [00:09<00:00,  8.25it/s]

Rendering models: 100%|██████████| 30/30 [00:09<00:00,  9.14it/s]

equidad1                              0.028457
Betelgeuse12                          0.000822
Thedunkmasta                          0.005048
citystars                             0.002086
not-logged-in-ab1612bc18cd7aab85db    0.000570
Linda_J._Berkel                       0.000421
not-logged-in-ea8ed61a7274cfb6a43b    0.809226
JEBala                                0.001660
SohaKawtharani                        0.002191
not-logged-in-fc6db6f8fca5399a40ba    0.000989
not-logged-in-9b2ae46779eb893a0ff9    0.017759
Paddy_van_CH                          0.000665
not-logged-in-a0164d241f30411408f0    0.000478
not-logged-in-c29ce12f02496eaffeb6    0.003105
GabeGramy                             0.001915
Snoopmatt                             0.013814
clars915                              0.000662
not-logged-in-5f30b2bb9b7f9d49da41    0.107500
databanana                            0.001075
aidenr                                0.001018
haydenmanninen                        0.015716
crush202020  

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())